In [2]:
# Notebook 출력설정
# 주요 라이브러리 임포트

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.grid'] = True
import seaborn as sns

import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request

import os
import time

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
model_ = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

In [216]:
#data-tga

df_ = pd.read_csv('tga.csv', encoding='utf-8')
df_.dropna(inplace=True)
df_ = df_[~df_['time_in_seconds'].str.contains(':')]
df_ = df_[~df_['time_in_seconds'].str.contains('text_message')]

df_['tis'] = df_['time_in_seconds'].apply(float)
df_['tis'] -= min(df_['tis'])

df_tga = pd.DataFrame()
df_tga['aut'] = df_['author.id']
df_tga['msg'] = df_['message']
df_tga['tis'] = df_['tis']

# df_[df_['time_in_seconds'].str.contains(':')]['time_in_seconds']
df_tga.info()
df_tga.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105946 entries, 1 to 108305
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   aut     105946 non-null  object 
 1   msg     105946 non-null  object 
 2   tis     105946 non-null  float64
dtypes: float64(1), object(2)
memory usage: 3.2+ MB


,aut,msg,tis
1,UCi7ZTmWUeUH5igELD-aTuvg,"THE GAME AWARDS 2021: Official Livestream Today, 7:30p ET / 4:30p PT / 12:30a GMT",0.0
2,UC1fxm9h8JAN24W7vAKxw7zA,RE Village GOTY,1.0
3,UCGw59eolmwA7c126brUFtsA,guys check ign,1.0
4,UCx5HjN20OpFz6AvMTCmPgeg,CAN'T WAIT FOR ALUL'S REWARD STOPPER DANCE THE VIDEO GAME YEEEEEET,1.0
5,UCeXuMmT_qB-NmtgKSm0tvqw,gex is what we need!,1.0


In [221]:
from collections import deque

df_ = df_tga

lst_author = df_['aut'].apply(str).values
lst_message = df_['msg'].apply(str).values
lst_tis = df_['tis'].values

lst_encode = model_.encode(lst_message, convert_to_numpy=True, show_progress_bar=True)

que_ = deque(zip(lst_author, lst_message, lst_tis, lst_encode))

Batches:   0%|          | 0/3311 [00:00<?, ?it/s]

In [ ]:
#options

SIZE_WINDOW = 500
MAX_VIEW = 10
THRESHOLD_ = .0

MAG_TIME = 1.

In [185]:

cnt_ = 0 
window_mes = ['' for _ in range(SIZE_WINDOW)]
window_mat = [[0 for _ in range(SIZE_WINDOW)] for _ in range(SIZE_WINDOW)]
window_tot = [0 for _ in range(SIZE_WINDOW)]

def window_insert():
    


In [182]:
def func_score(idx_a, idx_b):
    encode_a, encode_b = lst_encode[idx_a], lst_encode[idx_b]
    score_ = float(util.cos_sim(encode_a, encode_b)[0][0])
    score_ = max(0, score_)
    return score_

In [183]:
from IPython.display import clear_output
import time

time_s = time.time()

while que_:
    time_t = time.time() - time_s
    while que_ and que_[0][3] <= time_t:
        aut_, mes_, tis_, encode_ = que_.pop()
        window_insert(mes_)


SyntaxError: unexpected EOF while parsing (4034490306.py, line 5)

In [186]:
from IPython.display import clear_output

import time

for idx_round in range(len(lst_corpus)):
    clear_output(wait=True)
    print("round", idx_round)
    
    idx_window = idx_round % SIZE_WINDOW
    window_idx[idx_window] = idx_round
    for idx_window_sub in range(SIZE_WINDOW):
        idx_sub = window_idx[idx_window_sub]
        val_old = window_mat[idx_window][idx_window_sub]
        window_tot[idx_window_sub] -= val_old
        
        if idx_sub == -1:
            score_ = 0.0
        else:
            score_ = func_score(idx_, idx_sub)
        
        window_mat[idx_window][idx_window_sub] = score_
        window_mat[idx_window_sub][idx_window] = score_
        window_tot[idx_window] += score_
        
#     for idx_, each_ in enumerate(window_mat):
        
        
#         print(' '.join(map(lambda x:str(round(x, 2)), each_)), end=' ')
#         print('tot=' window_tot[idx_])
    
    lst_printed = [] 
    lst_c = [idx_ for _, idx_ in sorted(zip(window_tot, range(SIZE_WINDOW)), reverse=True)]
    for idx_window in lst_c:
        for idx_printed in lst_printed:
            if window_mat[idx_window][idx_printed] > THRESHOLD_:
                break
        else:
            idx_ = window_idx[idx_window]
            corpus_ = lst_corpus[idx_]
            print(corpus_, window_tot[idx_window])
            lst_printed.append(idx_window)
            if len(lst_printed) == MAX_VIEW:
                break

    time.sleep(.01)

round 2025


KeyboardInterrupt: 

In [ ]:
from sentence_transformers import util
time_s = time.time()
clusters_ = util.community_detection(encodes_[2000:4000],threshold=0.9, min_community_size=5, init_max_size=100)
print(time.time()-time_s)

In [59]:
str_ = df_['message'].apply(str).values

from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters,)
clustering_model.fit(encodes_[10000:12000])
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(str_[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  1
['hi', 'Hallo', 'hello', '흢훳k휌', 'Gang', 'nakamuraFallKaru', 'bro chess is so much more fun when you just flag people', '!candidates', 'nakamuraAha', 'how long to wait', 'Hikaru has a brand new TikTok! Keep up on highlights and clips at https://tiktok.com/@hikaruGM', '!shoes', "@JxA__ New sub goal! If we hit 8000 subs before The Candidates, Hikaru'll wear these ?뜊?몶 https://www.fillingpieces.com/collections/men-new-arrivals-footwear/products/loafer-ananas-black ?뵦 We're at 6667", 'good morning from snowwy Calgary', 'I might sub', 'peepoHey', 'ResidentSleeper', '@pataricky link your twitch account to your amazon prime account', '@pataricky Do you have an amazon prime account :)', 'hi hikaru', "I think I'm right", 'I am currently in class rn lol', 'And good morning everyone', 'morninggg', 'that was a sick overlay', 'MELEEEEEE !', 'pogger', '@benedict_jojo Thank you, kind sir.', 'pog', "@pataricky I bet you're not", '8 candidate is Ding Liren obviously', '@yoyoiyoo Hikaru has b

In [56]:


for cluster_ in clusters_:
    for each_ in cluster_:
        print(df_['message'].apply(str).values[each_])
    print()

Gang gang
good morning
i want to make friend with someone!
8:51
Poop
Click here for MASSIVE Elo Boost - also to sub to Hikaru on Youtube! https://www.youtube.com/c/gmhikaru?sub_confirmation=1
Good morning all from Toronto!
@pataricky He's not at his computer yet would be my guess
ResidentSleeper
@pataricky link your twitch account to your amazon prime account
@radar_cs  okay thanks for letting me know
Good morning
MUTEDddddddddddddddddd
Pog wtf was that
BOXerrrrrrrrrrrr
I'm gonna subscribe
Pog
Not playing e4 at all lepNOTED
!speedrun
!speedrun
then the rest of the openings that you wouldn't played would result to your opponent preparing for the rest, no?
yup
@gmhikaru do you prefer Rapport with long or short hair?
Polish Speed Run?
i like the reti for white but im struggling for finding a good opening for black. im hoping to be inspired in this stream
speedrun?
@milkymooose morning
caro kann noooo
...
is Na6 suppose to be bad on move 2
@unknownchessp1ayer This is the Botez Gambit Speed

In [50]:
for idx_, str_ in enumerate(lst_str):
    print(idx_,str_)

0 dumbass.
1 stupid boy.
2 you die!
3 I love you.
4 I like you.
5 I hate you.
6 I am sorry.
7 I apologize to you.
8 I will kill you.
9 lol
10 hahahahaha
11 hahahaha
12 suck it!
13 drink it.
14 My name is T.
15 I am T to you.


In [51]:
encodes_.shape

(16, 768)